In [14]:
import numpy as np
from numpy.testing import assert_almost_equal
import pandas as pd
from matplotlib import pyplot as plt


In [45]:
### Helper functions

def is_square(m):
    assert m.shape[0] == m.shape[1]

def covariance_correct(m, n_steps):
    assert m.shape[0] == n_steps


def covariance_calc(n_steps, T, H, k):
    #Calculation derived in dissertation. Absolute needed only for case where k = 0
    
    return 1/2 * ((T/n_steps) ** H)\
        * (abs(k - 1) ** (2 * H)\
         + (k + 1) ** (2 * H)\
         - 2 * (k ** (2 * H)))

def construct_covariance_matrix(n_steps, T, H):
    m = np.zeros([n_steps , n_steps])
    k = np.arange(0, n_steps, 1)
    m[:, 0] = covariance_calc(n_steps, T, H, k)

    for i in range(1, n_steps):
        m[i:, i] = m[i-1:-1, i-1]
    
    m = np.tril(m) + np.triu(m.T, 1)

    return m

def simulate_fbmn_sims, n_steps, mu, Sigma):
    is_square(Sigma)
    covariance_correct(Sigma, n_steps)


In [53]:
construct_covariance_matrix(1024, 1, 0.05)

array([[ 7.07106781e-01, -3.28177640e-01, -9.69576016e-03, ...,
        -6.10307627e-08, -6.09173503e-08, -6.08042594e-08],
       [-3.28177640e-01,  7.07106781e-01, -3.28177640e-01, ...,
        -6.11444976e-08, -6.10307627e-08, -6.09173503e-08],
       [-9.69576016e-03, -3.28177640e-01,  7.07106781e-01, ...,
        -6.12585565e-08, -6.11444976e-08, -6.10307627e-08],
       ...,
       [-6.10307627e-08, -6.11444976e-08, -6.12585565e-08, ...,
         7.07106781e-01, -3.28177640e-01, -9.69576016e-03],
       [-6.09173503e-08, -6.10307627e-08, -6.11444976e-08, ...,
        -3.28177640e-01,  7.07106781e-01, -3.28177640e-01],
       [-6.08042594e-08, -6.09173503e-08, -6.10307627e-08, ...,
        -9.69576016e-03, -3.28177640e-01,  7.07106781e-01]])

In [41]:
m = np.zeros([10 , 10])
print(m[10:, 10])

IndexError: index 10 is out of bounds for axis 1 with size 10

In [51]:
4 ** 0.1 + 6 ** 0.1 - 2 * 5 ** 0.1

-0.015444159555729087